In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

from sklearn.linear_model import  LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor


from statsmodels.tsa.statespace.sarimax import SARIMAXResults
from numpy.linalg.linalg import LinAlgError

from scipy import stats

from matplotlib import pyplot as plt
from statsmodels import api as sm
from tqdm import tqdm 
import warnings
from itertools import product

#### 1.Загрузите обучающие выборки прошлой недели, перечислите используемые в моделях признаки и посчитайте Qmay — качество прогнозов моделей, настроенных на данных до апреля 2016, в мае 2016.

In [2]:
df0=pd.read_csv('0.n_reg_2015-09_2016-06.csv',sep=',',parse_dates=['t'],index_col=['t'])
cols=df0.columns

In [3]:
time_start, time_end= '2015-11-01 00:00:00', '2016-06-30 23:00:00',
df_all=df0[time_start:time_end]

In [4]:
df_all.shape

(5832, 102)

Признаки, используемые на прошлой неделе:
1. Временные -- зависят от времени (синусы и косинусы)
2. Кумулятивные -- сумма поездок за период (4,8,12,24,24*7,24*30 часов)
3. Зональные временные - количество поездок из рассматриваемого района в предыдущие часы (смещение от 1-10)
4. Данные из ARIMA

In [5]:
#Функция формирует временные признаки (не зависящие от географической зоны)
def get_time_features(df, K):
    length = df.shape[0] + 1
    features = pd.DataFrame()
    for i in range(1,K+1):
        features['sinw_w%d' % i] = [ np.sin(t * i * 2.*np.pi / 168.) for t in np.arange(1, length)]
        features['cosw_w%d' % i] = [ np.cos(t * i * 2.*np.pi / 168.) for t in np.arange(1, length)]
        features['sind_%d' % i] = [ np.sin(t * i * 2.*np.pi / 24.) for t in np.arange(1, length)]
        features['cosd_%d' % i] = [ np.cos(t * i * 2.*np.pi / 24.) for t in np.arange(1, length)]

    features[['h%d' % i for i in range(0, 24)]] = pd.get_dummies(df.index.hour)
    features[['dw%d' % i for i in range(0, 7)]] = pd.get_dummies(df.index.weekday)
    features[['dm%d' % i for i in range(0, 31)]] = pd.get_dummies(df.index.day)
    features['is_weekend'] = (df.index.weekday > 4).astype(float)
      
    return features

In [6]:
#Кумулятивное скользящее
def mov_cum (z,N=3):
    cumsum, moving_cumes = [0], []
    for i, x in enumerate(z, 1):
        cumsum.append(cumsum[i-1] + x)
        if i>=N:
            moving_cum = (cumsum[i] - cumsum[i-N])
        
            moving_cumes.append(moving_cum)
            
    return [0]*(N-1)+moving_cumes    

In [7]:
#Зональные временные и кумулятивные регрессионные признаки
def get_trip_features(df,Region,K):
    L=df.shape[0]
    S=df[Region].values
    features = pd.DataFrame()
    y0=np.zeros(K)
    
    #количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K 
    for j in range(K):
        y=[S[i-j] for i in range(K,L)]
        
        features['n-_%d' % j]=np.concatenate((y0,y),axis=0)
                 
    #суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц                 
    hours=[4,8,12,24,24*7,24*30]  
    for h in hours:
        features['sum_%d' % h]=mov_cum(S,N=h)
    
    return  features        

In [8]:
df_ARIMA=pd.read_csv('ARIMA_pred(15.11-16.06)',',') #предсказания по модели ARIMA

Приготовим 6 пар обучающих регрессионных признаков размера X и ответов y

In [9]:
#Формирует df - регрессионные данные Х и ответы y. 6 вариантов ответов для шага прогноза (1,...6)
def get_Xy_data(df, Ktime = 20, Ktrip=10):
    
    Xtime=get_time_features(df, Ktime) #создание временных признаков

    data=[]
    
    #перебор по зонам для создания кумулятивных и зональных временных признаков
    for j,r in enumerate(cols): 
        #if j==1:
        #    break
        
        Xtrip=get_trip_features(df, r, Ktrip)
        X=pd.concat((Xtime,Xtrip), axis=1) # объединение временных признаков и зональных
        
        # добавление столбцов предсказаний ARIMA
        x_arima=df_ARIMA[str(r)]
        for i in range(1,7): 
            X['ARIMA_%d' % i]=x_arima.shift(-i).values
        
        if j==0:
            X_size=X.shape[1] # замер ширины таблицы регрессионных признаков
        
        # добавление 6 столбцов ответов:
        y=df_all[r]
        for i in range(1,7): 
            X['y_%d' % i]=y.shift(-i).values
        
        X.index=df.index
        
        #Обрезка 0 в начале и 6 строк в конце
        X=X.iloc[24*30-1:df.shape[0]-6] 
        data.append(X)
    return data, X_size

In [10]:
%%time
data, X_size =get_Xy_data(df_all, Ktime = 20, Ktrip=10)
print X_size

165
Wall time: 14.2 s


In [11]:
data[0].head(3)

,sinw_w1,cosw_w1,sind_1,cosd_1,sinw_w2,cosw_w2,sind_2,cosd_2,sinw_w3,cosw_w3,...,ARIMA_3,ARIMA_4,ARIMA_5,ARIMA_6,y_1,y_2,y_3,y_4,y_5,y_6
t,,,,,,,,,,,,,,,,,,,,,
2015-11-30 23:00:00,0.974928,-0.222521,6.862974e-15,1.000000,-0.433884,-0.900969,1.372595e-14,1.000000,-0.781831,0.623490,...,10.444599,3.255400,3.330129,3.999257,19.0,16.0,8.0,3.0,2.0,7.0
2015-12-01 00:00:00,0.965926,-0.258819,2.588190e-01,0.965926,-0.500000,-0.866025,5.000000e-01,0.866025,-0.707107,0.707107,...,3.255400,3.330129,3.999257,12.881411,16.0,8.0,3.0,2.0,7.0,20.0
2015-12-01 01:00:00,0.955573,-0.294755,5.000000e-01,0.866025,-0.563320,-0.826239,8.660254e-01,0.500000,-0.623490,0.781831,...,3.330129,3.999257,12.881411,24.737939,8.0,3.0,2.0,7.0,20.0,54.0


Из предыдущей недели выявлено, что среди моделей Лассо, Линейная регрессия, Случайный лес, лучший результат дает Линейнаяя регрессия, поэтому далее используется Линейная регрессия

In [12]:
#Составим предсказательные модели для каждой зоны, каждаго предсказательного шага смещения

#Линейная регрессия

def get_LinearRegr_models(data, X_size, t_end):
    
    models=[]
    for dat in data:
        X=dat[:t_end].iloc[:,:X_size]
        m=[]
        for j in range(6):
            y=dat[:t_end].iloc[:,X_size+j]
            m.append(LinearRegression().fit(X, y))
        models.append(m)
    return models

In [13]:
%%time
t_end='2016-04-30 17:00:00' # Граница обрезки данных
LR_models=get_LinearRegr_models(data, X_size,t_end) #Получение списка моделей по зонам

Wall time: 33.8 s


Посчитаем качество модели, настроенных на данных до апреля 2016, в мае 2016

In [14]:
#Список концов истории:
time_ends=df_all['2016-04-30 23:00:00':'2016-05-31 17:00:00'].index

In [15]:
def Qmay(data, X_size, models,time_ends):
    q=0
    for r in range(len(cols)): #преребор по районам
        dat=data[r]
        model=models[r]
        if r==10:
            break
   
        for i,t in enumerate(time_ends): #перебор по концам истории
            X=dat[t:t].iloc[:,:X_size]
        
            for j in range(6): #перебор по шагам прогноза
            
                m=model[j]
                y_hat=m.predict(X)
                y_real=dat[t:t].iloc[:,X_size+j]
                
                q+=np.abs(y_hat[0]-y_real[0])
            
    Qmay=1./(739*6)*q
    return Qmay

In [16]:
%%time
print 'Qmay:', Qmay(data, X_size,LR_models,time_ends)

Qmay: 41.60485644941389
Wall time: 1min 9s


#### 2.Попробуйте добавить признаки. Используйте идеи, которые мы предложили, или какие-то свои. Обучайте обновлённые модели на данных до апреля 2016 включительно и считайте качество новых прогнозов на мае. Удаётся ли вам улучшить качество? Не нужно ли увеличить сложность регрессионной модели? Если добавляемый признак не улучшает качество, всё равно оставьте доказательства этому в ноутбуке, чтобы ваши коллеги это видели при проверке.

Попробуем улучшить качество предсказания, заменив в ARIMA данных отрицательные значения нулями

In [17]:
for c in df_ARIMA.columns:
    df_ARIMA[c] = df_ARIMA[c].map(lambda x: x if x>=0 else 0)

In [18]:
%%time
data, X_size =get_Xy_data(df_all, Ktime = 20, Ktrip=10)
LR_models=get_LinearRegr_models(data, X_size, t_end)
print Qmay(data, X_size, LR_models,time_ends)

47.847295066264195
Wall time: 1min 48s


In [ ]:
#Вывод
#Обрезание отрицательных значений у ARIMA модели ухудшает точность предсказания

In [19]:
# вернемся к первоначальным данным ARIMA
df_ARIMA=pd.read_csv('ARIMA_pred(15.11-16.06)',',') #предсказания по модели ARIMA

In [ ]:
Попробуем улучшить прогноз, введя зональные временные и кумулятивные признаки по сдедующим данным:
1)'passenger_count'
2)'trip_distance'
3)'fare_amount'
4)'tolls_amount'
5)'total_amount'

Для них предварительно подготовлены файлы данных, аггрегированных по часам и регионам, аналогично тому, как это было сделано для числа поездок

In [20]:
time_start, time_end= '2015-11-01 00:00:00', '2016-06-30 23:00:00',

a_0=df_all #Данные по числу поездок
a_1=pd.read_csv('1.pass_count_2015-09_2016-06.csv',sep=',',parse_dates=['t'],index_col=['t'])[time_start:time_end]
a_2=pd.read_csv('2.trip_dist_2015-09_2016-06.csv',sep=',',parse_dates=['t'],index_col=['t'])[time_start:time_end]
a_3=pd.read_csv('3.fare_am_2015-09_2016-06.csv',sep=',',parse_dates=['t'],index_col=['t'])[time_start:time_end]
a_4=pd.read_csv('4.tolls_am_2015-09_2016-06.csv',sep=',',parse_dates=['t'],index_col=['t'])[time_start:time_end]
a_5=pd.read_csv('5.total_am_2015-09_2016-06.csv',sep=',',parse_dates=['t'],index_col=['t'])[time_start:time_end]


In [ ]:
Была попытка сформировать данные по категориальным признакам, таким как 
    наиболее популярный: район доставки,
                            'VendorID'
                            'RatecodeID'
                            'payment_type'

Однако вычислительное время с использованием функции stats.binned_statistic_2d оказалось слишком большим на моем ноутбуке, 
так что пришлось оказаться от вычисления этих признаков 

Модифицируем функцию, формирующую регрессионные признаки X с учетом аггрегаций по другим исходным признакам и 6 ответов, соответствующих шагам прогноза

In [21]:
#Формирует df - регрессионные данные Х и ответы y. 6 вариантов ответов для шага прогноза (1,...6)
def get_Xy_data_on_dfs(dfs, Ktime, Ktrip):
    
    data=[]
    
    Xtime=get_time_features(dfs[0], Ktime) #создание общих временных признаков
   
    #перебор по зонам для создания кумулятивных и зональных временных признаков
    for j,r in enumerate(cols): 
        X=Xtime
        
        #Перебор по аггрегациям  
        for k, df in enumerate(dfs): 
            Xtrip=get_trip_features(dfs[k], r, Ktrip) 
            X=pd.concat((X,Xtrip), axis=1) # объединение временных признаков и зональных
           
       # добавление столбцов предсказаний ARIMA
        x_arima=df_ARIMA[str(r)]
        for i in range(1,7): 
            X['ARIMA_%d' % i]=x_arima.shift(-i).values
        
        if j==0:
            X_size=X.shape[1] # замер ширины таблицы регрессионных признаков
        
        # добавление 6 столбцов ответов:
        y=df_all[r]
        for i in range(1,7): 
            X['y_%d' % i]=y.shift(-i).values
        
        X.index=df.index
        
        #Обрезка 0 в начале и 6 строк в конце
        X=X.iloc[24*30-1:df.shape[0]-6] 
        data.append(X)
    return data, X_size

In [22]:
#Функция для рассчета Qmay по нескольким аггрегированным датафреймам
def calculate_Qmay(dfs, Ktime = 20, Ktrip=10):
    data, X_size =get_Xy_data_on_dfs(dfs, Ktime, Ktrip)
    LR_models=get_LinearRegr_models(data, X_size, t_end)
    #print Ktime,Ktrip,X_size
    return Qmay(data, X_size, LR_models, time_ends)

В начале исследуем одиночное введение признаков

In [23]:
%%time
print 'Qmay for n_trips :', calculate_Qmay([a_0],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+passenger_count :', calculate_Qmay([a_0,a_1],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+trip_distance :', calculate_Qmay([a_0,a_2],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+fare_amount :', calculate_Qmay([a_0,a_3],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+tolls_amount :', calculate_Qmay([a_0,a_4],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+total_amount :', calculate_Qmay([a_0,a_5],Ktime = 20, Ktrip=10)

Qmay for n_trips : 41.60485644941389
Qmay for n_trips+passenger_count : 42.10840551246281
Qmay for n_trips+trip_distance : 74.39815389270112
Qmay for n_trips+fare_amount : 42.211517769091984
Qmay for n_trips+tolls_amount : 42.148516467663974
Qmay for n_trips+total_amount : 42.17946374433456
Wall time: 11min 4s


In [ ]:
Вывод:
Введение дополнительных признаков ухудшает предсказание. Особенно ухудшает введение признака 'trip_distance'


In [ ]:
Введем все признаки, кроме второго -- 'trip_distance'
И все вместе со вторым

In [24]:
%%time
print 'Qmay for all without trip_distance:', calculate_Qmay([a_0,a_1,a_3,a_4,a_5])
print 'Qmay for all:', calculate_Qmay([a_0,a_1,a_2,a_3,a_4,a_5])

Qmay for all without trip_distance: 44.42907579512345
Qmay for all: 167.1239350510453
Wall time: 5min 18s


In [ ]:
Вывод: введение признаков
    'passenger_count', 'trip_distance', 'fare_amount', 'tolls_amount', 'total_amount'
не улучшает предсказание

In [ ]:
Попробуем увеличить параметры Ktime = 20, Ktrip=10

In [ ]:
Проверим, как влияет увеличение Ktime, Ktrip на введение других аггрегированных данных

In [26]:
%%time
print calculate_Qmay([a_0],Ktime = 30, Ktrip=30)
print calculate_Qmay([a_0,a_1],Ktime = 30, Ktrip=30)
print calculate_Qmay([a_0,a_2],Ktime = 30, Ktrip=30)
print calculate_Qmay([a_0,a_3],Ktime = 30, Ktrip=30)
print calculate_Qmay([a_0,a_4],Ktime = 30, Ktrip=30)
print calculate_Qmay([a_0,a_5],Ktime = 30, Ktrip=30)

38.80779586367657
39.374513775262955
80.98800265079616
39.675479758858735
39.42209736284507
39.75138958911764
Wall time: 19min 39s


In [ ]:
Вывод: 
Увеличение Ktime, Ktrip не делает введение других аггрегированных данных полезным для точности предсказания

In [ ]:
Проанализируем введение в модель регрессионного признака наличия праздников
Признак имеет вид столбца, принимающего значение 1 в строках, соответствующих праздничным дням и 0 во всех остальных днях

In [27]:
USA_holidays=['11.11.2015', 
'24.12.2015', 
'25.12.2015', 
'31.12.2015', 
'01.01.2016', 
'19.01.2016', 
'14.02.2016', 
'17.03.2016', 
'27.03.2016', 
'28.03.2016', 
'30.05.2016']

In [28]:
# Преобразуем к стандартному временному формату и обрежем часы
from datetime import datetime
US_h=[str(datetime.strptime(x,'%d.%m.%Y'))[:10] for x in USA_holidays]
US_h

['2015-11-11',
 '2015-12-24',
 '2015-12-25',
 '2015-12-31',
 '2016-01-01',
 '2016-01-19',
 '2016-02-14',
 '2016-03-17',
 '2016-03-27',
 '2016-03-28',
 '2016-05-30']

Модернизируем функцию 'get_time_features', так, чтобы внутри нее к временным признакам добавлялся признак наличия праздника

In [29]:
def get_time_features(df, K):
    length = df.shape[0] + 1
    features = pd.DataFrame()
    for i in range(1,K+1):
        features['sinw_w%d' % i] = [ np.sin(t * i * 2.*np.pi / 168.) for t in np.arange(1, length)]
        features['cosw_w%d' % i] = [ np.cos(t * i * 2.*np.pi / 168.) for t in np.arange(1, length)]
        features['sind_%d' % i] = [ np.sin(t * i * 2.*np.pi / 24.) for t in np.arange(1, length)]
        features['cosd_%d' % i] = [ np.cos(t * i * 2.*np.pi / 24.) for t in np.arange(1, length)]

    features[['h%d' % i for i in range(0, 24)]] = pd.get_dummies(df.index.hour)
    features[['dw%d' % i for i in range(0, 7)]] = pd.get_dummies(df.index.weekday)
    features[['dm%d' % i for i in range(0, 31)]] = pd.get_dummies(df.index.day)
    features['is_weekend'] = (df.index.weekday > 4).astype(float)
   
    # Вычисление признака наличия праздника
    hh=[]
    for x in df.index:
        if str(x)[:10] in US_h:
            h=1
        else:
            h=0
        hh.append(h)
    feature=pd.DataFrame(index=df.index)
    features['is_h']=hh
        
    return features

In [30]:
%%time
print 'Qmay for n_trips :', calculate_Qmay([a_0],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+passenger_count :', calculate_Qmay([a_0,a_1],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+trip_distance :', calculate_Qmay([a_0,a_2],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+fare_amount :', calculate_Qmay([a_0,a_3],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+tolls_amount :', calculate_Qmay([a_0,a_4],Ktime = 20, Ktrip=10)
print 'Qmay for n_trips+total_amount :', calculate_Qmay([a_0,a_5],Ktime = 20, Ktrip=10)

Qmay for n_trips : 41.63245213930221
Qmay for n_trips+passenger_count : 42.08893290643621
Qmay for n_trips+trip_distance : 73.57846512743863
Qmay for n_trips+fare_amount : 42.23561227941782
Qmay for n_trips+tolls_amount : 42.153387513041245
Qmay for n_trips+total_amount : 42.16784896270351
Wall time: 11min 37s


In [ ]:
Тот же рассчет без добавления признака наличия праздника давал результаты:

In [ ]:
41.60485644941389 # анализировалось только количество поездок
42.10840551246281 # к-во поездок + 'passenger_count'
74.39815389270112 # к-во поездок + 'trip_distance'
42.211517769091984 # к-во поездок + 'fare_amount'
42.148516467663974 # к-во поездок + 'tolls_amount'
42.17946374433456 # к-во поездок + 'total_amount'

In [ ]:
Выводы:
Учет праздников положительно влияет на:
    анализ по количеству поездок
    анализ по к-ву поездок + 'passenger_count'
    анализ по к-ву поездок + 'fare_amount'
    анализ по к-ву поездок + 'total_amount'

Учет праздников отрицательно влияет на:    
    анализ по к-ву поездок + 'trip_distance'
    анализ по к-ву поездок + 'tolls_amount'
Вероятно это связано с наличием корреляции между празниками и этими признаками

Введение признака праздников не делает полезным введение в модель основанную на анализе числа поездок 
дополнительных признаков

Качество предсказания существенно улучшается только при увеличении параметров Ktime, Ktrip и 
незначительно при учете праздников

In [ ]:
Рассмотрим динамику улучшения предсказания в зависимости от параметров Ktime и Ktrip с учетом праздничных дней

In [31]:
%%time
print 'Ktime = 10, Ktrip=5 :', calculate_Qmay([a_0],Ktime = 10, Ktrip=5)
print 'Ktime = 20, Ktrip=10 :',calculate_Qmay([a_0],Ktime = 20, Ktrip=10)
print 'Ktime = 30, Ktrip=20 :',calculate_Qmay([a_0],Ktime = 30, Ktrip=20)
print 'Ktime = 30, Ktrip=30 :',calculate_Qmay([a_0],Ktime = 30, Ktrip=30)
print 'Ktime = 60, Ktrip=40 :',calculate_Qmay([a_0],Ktime = 60, Ktrip=40)
print 'Ktime = 120, Ktrip=80 :',calculate_Qmay([a_0],Ktime = 120, Ktrip=80)

Ktime = 10, Ktrip=5 : 45.28610286443238
Ktime = 20, Ktrip=10 : 41.63245213930221
Ktime = 30, Ktrip=20 : 39.473990471036196
Ktime = 30, Ktrip=30 : 38.860910261502475
Ktime = 60, Ktrip=40 : 37.04725447285182
Ktime = 120, Ktrip=80 : 37.81591809797042
Wall time: 17min 44s


In [ ]:
Вывод: 
при уменьшении величины Ktime и Ktrip точность падает
при увеличении величины Ktime и Ktrip точность растет, но до определенного предела и далее падает

In [ ]:
При расчете предсказанных значений появляются отрицательные величины. Их можно заменить:
    а) 0 (-y => 0)
    б) модулем этой величины (-y => |y|)

In [ ]:
Для проверки гипотезы модифицируем функции:

In [43]:
def Qmay_ab(data, X_size, models,time_ends):
    q, q0, q1 =0.,0.,0.
    for r in range(len(cols)): #преребор по районам
        dat=data[r]
        model=models[r]
        #if r==10:
            #break
   
        for i,t in enumerate(time_ends): #перебор по концам истории
            X=dat[t:t].iloc[:,:X_size]
        
            for j in range(6): #перебор по шагам прогноза
            
                m=model[j]
                y_hat=m.predict(X)
                y_real=dat[t:t].iloc[:,X_size+j]
                
                #модификация - замена предсказанного отрицательного y на 0
                if y_hat[0]<0:
                    y_hat0=[0]
                    y_hat1=[np.abs(y_hat[0])]
                else:
                    y_hat0, y_hat1= y_hat, y_hat
                
                q+=np.abs(y_hat[0]-y_real[0])
                q0+=np.abs(y_hat0[0]-y_real[0])
                q1+=np.abs(y_hat1[0]-y_real[0])
            
    Qmay=1./(739*6)*q
    Qmay0=1./(739*6)*q0
    Qmay1=1./(739*6)*q1
    return Qmay, Qmay0, Qmay1

In [44]:
def calculate_Qmay_ab(dfs, Ktime = 20, Ktrip=10):
    data, X_size =get_Xy_data_on_dfs(dfs, Ktime, Ktrip)
    LR_models=get_LinearRegr_models(data, X_size, t_end)
    #print Ktime,Ktrip,X_size
    return Qmay_ab(data, X_size, LR_models, time_ends)

In [45]:
%%time
print 'Ktime = 30, Ktrip=30 : Qmay, Qmay a), Qmay б):',calculate_Qmay([a_0],Ktime = 30, Ktrip=30)

Ktime = 30, Ktrip=30 : Qmay, Qmay a), Qmay б): (38.860910261502475, 38.82103180962501, 38.80646969451088)
Wall time: 2min 25s


In [47]:
%%time
print 'Ktime = 60, Ktrip=40 : Qmay, Qmay a), Qmay б):',calculate_Qmay([a_0],Ktime = 60, Ktrip=40)

Ktime = 60, Ktrip=40 : Qmay, Qmay a), Qmay б): (37.04725447285182, 36.99424017553102, 36.975460840909854)
Wall time: 3min 53s


In [48]:
%%time
print 'Ktime = 40, Ktrip=40 : Qmay, Qmay a), Qmay б):',calculate_Qmay([a_0],Ktime = 40, Ktrip=40)

Ktime = 40, Ktrip=40 : Qmay, Qmay a), Qmay б): (37.53244058460162, 37.4977671303293, 37.47720847554877)
Wall time: 3min 1s


In [ ]:
Вывод:
    Замена предсказанных отрицательных значений их модулями улучшает качество предсказания

In [ ]:
Остановимся на следующих подходах:
    Используются только данные числа поездок
    Учитываются праздничные дни
    Используются регрессионные признаки, соответствующие Ktime = 30, Ktrip=40
    Отрицательные предсказания заменяются их модулями

#### 3. Когда вы примете решение остановиться и перестать добавлять признаки, постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

In [ ]:
# Qmay было:
41.60485644941389

# Qmay стало: 
37.04725447285182

In [ ]:
Вывод: Точность повысилась

#### 4. Переобучите итоговые модели на данных до мая 2016 включительно, постройте прогнозы на июнь для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в уже знакомом вам формате: geoID, histEndDay, histEndHour, step, y

In [53]:
%%time
time_ends=df_all['2016-05-31 23:00:00':'2016-06-30 17:00:00'].index
t_end='2016-05-31 17:00:00'

Wall time: 11 ms



In [55]:
%%time
data, X_size =get_Xy_data_on_dfs([a_0], 60, 40)
models=get_LinearRegr_models(data, X_size, t_end)

Q=[]
for r in range(len(cols)): #преребор по районам
    dat=data[r]
    model=models[r]
    i_zone=cols[r]
    #if r==1:
        #break
        
    for i,t in enumerate(time_ends): #перебор по концам истории
        X=dat[t:t].iloc[:,:X_size]
        
        for j in range(6): #перебор по шагам прогноза
            
            m=model[j]
            y_hat=m.predict(X)[0]
            
            te=time_ends[i]
            id_='{}_2016-{:02}-{:02}_{}_{}'.format(i_zone, te.month, te.day, te.hour, j+1)
            
            Q.append([id_, y_hat])

Predict_june=pd.DataFrame(Q)

Wall time: 8min 56s


#### 5. Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

In [57]:
Predict_june.to_csv('Kaggle3.csv',',',index=False)

Ссылка на сабмишн https://inclass.kaggle.com/c/yellowtaxi/leaderboard#score
    Score 51.86995   